In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Nsites = 16 # user input, number of sites for 1 ML on model surface
npoints = 100 # user input, more points -> smoother transition lines between phases
T = np.linspace(200,1000,npoints) # user input, absolute temperature (K)
P = np.logspace(-7,7,npoints) # user input, pressure (Pa)

## inputs from DFT, note len(n) must equal len(Eads) ##
n = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,
     15,16,17,18,20,24,28,32] # user input, number of adsorbates on surface for each model
Eads = [-0.4402225,-0.871558,-1.2943415,-1.706719,-2.0884505,-2.470163,-2.8398205,-3.206556,-3.5498435,-3.885031,
        -4.1943495,-4.493219,-4.7885505,-5.079798,-5.3424835,-5.604954,-5.6745315,-5.694483,-5.561533,-4.776272,
        -1.898689,1.767797] # user input, average adsorption energy for each model

# calculate Gibbs free adsorption energies for all T, P, and n
Gtotal = []
for item in T:
    for item2 in P:
        ii = 0
        for item3 in n:
            ## Gas phase free energy calculations ##
            ## Values shown for H2(g) and must be adjusted based on the desired molecule ##
            # gas phase vibrational partition function
            hv = 537.459121*10**-3 # eV, H2 vibrational frequency multiplied by Planck's constant
            kB = 8.617333262*10**-5 # eV/K, Boltzmann's constant
            qvib = (np.exp(-hv/(2*kB*item))/(1-np.exp(-hv/(kB*item))))
            Gvib = -kB*item*np.log(qvib) # eV, gas phase vibrational energy

            # gas phase rotational partition function
            mH2 = (2.016*10**-3)/(6.0221408*10**23) #kg/molecule, molar mass of gas phase H2
            μH2 = mH2/4 # kg, reduced mass for linear H2 molecule
            R = 0.7475035173*10**-10 # m, distance between both atoms in H2
            I = μH2*R**2 # kg*m^2/molecule, moment of inertia
            kb = 1.380649*10**-23 # J/K, Boltzmann's constant
            h = 6.62607015*10**-34 # J*s, Planck's constant
            σ = 2 # symmetry number
            qrot = 8*(np.pi**2)*I*kb*item/(σ*h**2)
            Grot = -kB*item*np.log(qrot) # eV, gas phase rotational energy

            # gas phase translational partition function
            qtrans = ((2*np.pi*mH2*kb*item/h**2)**1.5)*(kb*item/item2)
            Gtrans = -kB*item*np.log(qtrans) # eV, gas phase translational energy

            ## Adsorbate-surface vibration energy calculations ##
            ## Values shown for H*/Pt(111) and must be adjusted based on the desired adsorbate/surface system ##
            θ = item3/Nsites # ML, coverage
         
            # Multi-variable regression model for calculating surface vibrational Gibbs free energy with coverage and T
            Gsurf = ((-0.00000228442*θ) - 0.00000002583573)*item**2 + ((0.0002917589*θ) - 0.0001180756)*item + ((3.088247*θ) - 0.0006232575)
                
            # save total Gibbs free adsorption energy 
            Gtotal.append(Eads[ii] + Gsurf - ((item3/2)*(Gvib + Grot + Gtrans)))
            ii = ii + 1

# finding coverage with lowest Gibbs free adsorption energy at each T and P to build 2D contour phase diagram
out_cov = []
ncount = 0
Pcount = 0
ncov = len(n)
for jj in range(0,npoints): # T
    for kk in range(0,npoints): # P
        Gref = 10**5
        for ii in range(0,ncov): # n
            if Gtotal[jj+kk+ii+ncount*(ncov-1)+Pcount*(npoints-1)] < Gref:
                Gref = Gtotal[jj+kk+ii+ncount*(ncov-1)+Pcount*(npoints-1)]
                if Gtotal[jj+kk+ii+ncount*(ncov-1)+Pcount*(npoints-1)] < 0:
                    save_cov = n[ii]
                else:
                    save_cov = 0
        ncount = ncount + 1
        out_cov.append(save_cov)
    Pcount = Pcount + 1

# generate 2D contour phase diagram
Z = np.array(out_cov).reshape(npoints,npoints)
lines = np.unique(Z)
print("All unique phases found: ",lines)
P1 = [np.log10(item*10**(-5)) for item in P]
X, Y = np.meshgrid(P1,T)
fig, ax = plt.subplots(1,1,figsize=(8,7))
CS = ax.contourf(X, Y, Z,(len(lines)+50),cmap='RdGy',vmin=min(lines),vmax=max(lines))
contours = ax.contour(X,Y,Z,lines,colors="black",linewidths=2,linestyles="solid")
ax.xaxis.set_tick_params(width=2,labelsize=12)
ax.yaxis.set_tick_params(width=2,labelsize=12)
ax.get_xaxis().set_visible(True)
ax.get_yaxis().set_visible(True)
ax.spines['top'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)
ax.set_xlabel("log[Pressure (bar)]",fontsize=16)
ax.set_ylabel("Temperature (K)",fontsize=16)
plt.tight_layout()
plt.show()